In [21]:
# bring in packages
import requests
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [8]:
# read tickers into a list
with open("tickers.txt", "r") as file:
    tickers = [ticker.strip() for ticker in file]
    
print(tickers)


['SOUN', 'MARA', 'CAVA', 'SPOT', 'OXY', 'BSEM', 'RKLB', 'RKT', 'WULF', 'CART', 'RUM', 'INSG', 'BTMD', 'CAE', 'CHGG', 'CNNE', 'CRVS', 'DAVE', 'DHT', 'DOX', 'GRWG', 'HNST', 'MODG', 'ORGO', 'PAY', 'SDRL', 'STNE', 'SURG', 'SWKS', 'VYGR', 'WPRT', 'ZI', 'AHR', 'AIRG', 'AMBC', 'AMPS', 'BLTE', 'BODI', 'DNA', 'EPM', 'EVCM', 'FIHL', 'GRDN', 'GUTS', 'HNRG', 'HPP', 'IAS', 'LGMK', 'LNW', 'LYV', 'MNTK', 'MRCC', 'MVST', 'SU', 'ZVRA', 'ALTG', 'BLFS', 'BW', 'CATX', 'CLSD', 'CRNX', 'CWD', 'DHX', 'DSP', 'EGAN', 'FGEN', 'FNA', 'HYPR', 'ICUI', 'LEGH', 'LXRX', 'NATL', 'NGL', 'NOTE', 'NPCE', 'NRGV', 'NTRA', 'OABI', 'OPRT', 'PGNY', 'PIII', 'PRDO', 'PSQH', 'PTVE', 'PUBM', 'PXLW', 'RBOT', 'RKDA', 'RMR', 'RPAY', 'RWAY', 'RXT', 'SCOR', 'SHLS', 'SIBN', 'SMTI', 'SMWB', 'SNCR', 'SND', 'SONXD', 'SPRY', 'SSTI', 'SVCO', 'TBPH', 'TTGT', 'TWFG', 'VTSI', 'ACNT', 'AENT', 'BTTR', 'BZFD', 'ELMD', 'GRAL', 'GROV', 'IRIX', 'KITT', 'KODK', 'LIDR', 'MAPS', 'ME', 'NTWK', 'OCX', 'PALT', 'PLBY', 'RLGT', 'SHFS', 'SLND', 'SNT', 'ULY',

In [18]:
# main processing of data
from json import JSONDecodeError


dataList = []
total_tickers = len(tickers)

for index, ticker in enumerate(tickers):
    print(f"Pulling data for ({index + 1}/{total_tickers}): {ticker}")
    r = requests.get(f"https://tr-cdn.tipranks.com/bff/prod/stock/{ticker}/payload.json")
    try:
        payloadData = r.json()["earning"]
        if payloadData["isHaveData"]:
            earningsPriceChangesList = payloadData["priceChanges"]
            earningsPriceChanges = []
            for epc in earningsPriceChangesList:
                if epc["change"] is not None:
                    earningsPriceChanges.append(round(epc["change"], 3))
                else:
                    print(f"No data for {ticker}")
            dataList.append({"ticker": ticker, "earningsPriceChanges": earningsPriceChanges})
    except JSONDecodeError:
        print(f"Error pulling data for {ticker}")

dataframe = pd.DataFrame(dataList)
print(dataframe.head(5))

Pulling data for (1/369): SOUN
Pulling data for (2/369): MARA
Pulling data for (3/369): CAVA
Pulling data for (4/369): SPOT
Pulling data for (5/369): OXY
Pulling data for (6/369): BSEM
Pulling data for (7/369): RKLB
Pulling data for (8/369): RKT
Pulling data for (9/369): WULF
Pulling data for (10/369): CART
Pulling data for (11/369): RUM
Pulling data for (12/369): INSG
Pulling data for (13/369): BTMD
Pulling data for (14/369): CAE
Pulling data for (15/369): CHGG
Pulling data for (16/369): CNNE
Pulling data for (17/369): CRVS
Pulling data for (18/369): DAVE
Pulling data for (19/369): DHT
Pulling data for (20/369): DOX
Pulling data for (21/369): GRWG
Pulling data for (22/369): HNST
Pulling data for (23/369): MODG
Pulling data for (24/369): ORGO
Pulling data for (25/369): PAY
Pulling data for (26/369): SDRL
Pulling data for (27/369): STNE
Pulling data for (28/369): SURG
Pulling data for (29/369): SWKS
Pulling data for (30/369): VYGR
Pulling data for (31/369): WPRT
Pulling data for (32/369

In [23]:
# assemble average price changeColumn
dataframe['averagePriceChange'] = dataframe['averagePriceChange'].apply(
    lambda x: f"{round(x * 100, 3)}%" if x is not None else None
)

print(dataframe.head())

  ticker  \
0   SOUN   
1   MARA   
2   CAVA   
3   SPOT   
4    OXY   

                                                                   earningsPriceChanges  \
0            [-0.065, 0.072, -0.186, 0.035, 0.035, -0.035, -0.219, -0.042, 0.025, 0.02]   
1  [-0.052, -0.127, -0.165, 0.08, -0.013, -0.146, 0.071, -0.035, -0.022, -0.105, 0.031]   
2                                                   [0.196, 0.07, 0.122, -0.077, 0.012]   
3       [0.12, 0.114, 0.039, 0.104, -0.143, 0.051, 0.127, -0.13, 0.122, -0.124, -0.168]   
4   [0.043, -0.021, 0.049, 0.016, 0.012, -0.036, -0.007, -0.092, -0.063, 0.012, -0.005]   

  averagePriceChange  
0              7.34%  
1               7.7%  
2              9.54%  
3            11.291%  
4             3.236%  


In [24]:
# export dataframe to csv
dataframe.to_csv('earningReportsData.csv', index=False)